In [2]:
from pymatgen.io.lammps.data import *

## Looks like I interpreted the special bond counts keyword incorrectly, it's counting the number of those bonds, not the number of atoms participating in those bonds. This counting method only works for the bonds. Need to see how many angles and dihedrals END with each atom

In [23]:
system = LammpsData.from_file("data.montecarlo_test")
xlig_dat = LammpsData.from_file("data.xligand")
oleic_dat = LammpsData.from_file("data.oleic_acid")

xlig = xlig_dat.structure
oleic = oleic_dat.structure

In [16]:
box, ff, tops = test.disassemble()

In [100]:
def generate_neighbors(topology,structure):
    neighbors = {atom: set() for atom in range(len(structure))}
    for relation in topology:
        for i in range(len(relation)):
            for j in range(i+1, len(relation)):
                neighbors[relation[i]].add(relation[j])
                neighbors[relation[j]].add(relation[i])
    return neighbors

In [103]:
topologies = ['Bonds', 'Angles', 'Dihedrals', 'Impropers']
neighbors = {topology: generate_neighbors(tops[0].topologies[topology], xlig) for topology in topologies}

In [115]:
neighbors["one_four"] = {at: neighbors['Dihedrals'][at].union(neighbors['Impropers'][at]) for at in neighbors['Dihedrals']}

In [122]:
special_bond_counts = {atom: [] for atom in range(len(xlig))}
special_bonds = {atom: set() for atom in range(len(xlig))}
for top in ['Bonds', 'Angles', 'one_four']:
    for atom in special_bond_counts:
        special_bond_counts[atom].append(len(neighbors[top][atom]))
        special_bonds[atom] = special_bonds[atom].union(neighbors[top][atom])

In [129]:
special_bond_counts

{0: [4, 10, 13],
 1: [4, 10, 13],
 2: [4, 9, 15],
 3: [3, 11, 19],
 4: [3, 9, 18],
 5: [3, 7, 12],
 6: [3, 7, 12],
 7: [3, 9, 14],
 8: [3, 7, 15],
 9: [1, 3, 7],
 10: [3, 9, 14],
 11: [3, 7, 12],
 12: [3, 7, 11],
 13: [3, 8, 14],
 14: [2, 7, 14],
 15: [4, 8, 13],
 16: [4, 10, 14],
 17: [4, 10, 16],
 18: [4, 10, 16],
 19: [4, 10, 16],
 20: [4, 10, 16],
 21: [4, 10, 16],
 22: [4, 10, 16],
 23: [4, 10, 14],
 24: [4, 8, 11],
 25: [2, 5, 8],
 26: [3, 7, 11],
 27: [3, 7, 12],
 28: [3, 7, 12],
 29: [3, 7, 12],
 30: [4, 9, 15],
 31: [1, 4, 9],
 32: [1, 4, 9],
 33: [1, 4, 9],
 34: [1, 4, 9],
 35: [1, 4, 8],
 36: [1, 4, 8],
 37: [1, 3, 7],
 38: [1, 3, 7],
 39: [1, 3, 7],
 40: [1, 3, 7],
 41: [1, 4, 7],
 42: [1, 4, 7],
 43: [1, 4, 9],
 44: [1, 4, 9],
 45: [1, 4, 9],
 46: [1, 4, 9],
 47: [1, 4, 9],
 48: [1, 4, 9],
 49: [1, 4, 9],
 50: [1, 4, 9],
 51: [1, 4, 9],
 52: [1, 4, 9],
 53: [1, 4, 9],
 54: [1, 4, 9],
 55: [1, 4, 9],
 56: [1, 4, 9],
 57: [1, 4, 9],
 58: [1, 4, 9],
 59: [1, 4, 7],
 60: [1, 4

In [107]:
for at in neighbors['Dihedrals']:
    if neighbors['Impropers'][at] != set():
        print(neighbors['Impropers'][at], neighbors["Dihedrals"][at])

{29, 4, 5} {0, 1, 2, 4, 5, 6, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 65, 66, 67}
{65, 3, 37, 5, 6, 28, 29} {0, 1, 2, 3, 35, 5, 37, 36, 6, 38, 7, 64, 65, 66, 67, 28, 29, 30}
{3, 4, 37, 6, 7, 38, 29} {65, 2, 3, 4, 37, 38, 6, 7, 8, 28, 29, 30}
{4, 37, 5, 38, 7, 8, 28} {64, 3, 4, 5, 38, 7, 8, 9, 10, 37, 28, 29}
{64, 5, 38, 6, 8, 9, 10, 28, 29} {64, 65, 4, 5, 6, 37, 8, 9, 10, 11, 38, 27, 28, 29}
{6, 7, 9, 10, 11, 27, 28} {64, 5, 6, 7, 38, 9, 10, 11, 12, 39, 26, 27, 28, 29, 63}
{8, 10, 7} {6, 7, 8, 10, 11, 27, 28}
{7, 8, 9, 39, 11, 12, 26, 27, 63} {6, 7, 8, 39, 9, 11, 12, 13, 40, 26, 27, 28, 62, 63}
{39, 8, 40, 10, 12, 13, 27} {39, 8, 7, 10, 9, 12, 13, 14, 40, 26, 27, 63}
{39, 40, 10, 11, 13, 14, 26} {39, 8, 40, 10, 11, 13, 14, 15, 26, 27, 62}
{40, 11, 12, 14, 26, 27, 62} {39, 40, 41, 10, 11, 12, 42, 14, 15, 16, 26, 27, 62, 63}
{26, 12, 13} {40, 41, 42, 11, 12, 13, 43, 15, 16, 17, 44, 26, 27, 62}
{10, 12, 13, 14, 27, 62, 63} {8, 40, 10, 11, 12, 13, 14, 15, 27, 62, 63}
{8, 10, 11, 13, 26, 62

In [111]:
for i in neighbors['one_four']:
    if neighbors['one_four'][i] != neighbors["Dihedrals"][i]:
        print(neighbors['one_four'][i], neighbors["Dihedrals"][i])

TypeError: list indices must be integers or slices, not set

In [79]:
special_bond_counts = []



In [57]:
bond_neighbors = {atom: set() for atom in range(len(xlig))}
for bond in tops[0].topologies['Bonds']:
    bond_neighbors[bond[0]].add(bond[1])
    bond_neighbors[bond[1]].add(bond[0])

In [86]:
# rob's way
%%time
angle_neighbors = {atom: set() for atom in range(len(xlig))}
for angle in tops[0].topologies['Angles']:
    for atom in angle:
        for i in range(len(angle)):
            if atom != angle[i]:
                angle_neighbors[atom].add(angle[i])

CPU times: user 302 µs, sys: 51 µs, total: 353 µs
Wall time: 357 µs


In [87]:
#chatgpt way
%%time
test_neighbors = {atom: set() for atom in range(len(xlig))}
for angle in tops[0].topologies['Angles']:
    for i in range(len(angle)):
        for j in range(i+1, len(angle)):
            test_neighbors[angle[i]].add(angle[j])
            test_neighbors[angle[j]].add(angle[i])

CPU times: user 182 µs, sys: 30 µs, total: 212 µs
Wall time: 215 µs


In [93]:
%%time
dihedral_neighbors = {atom: set() for atom in range(len(xlig))}
for dihedral in tops[0].topologies['Dihedrals']:
    for i in range(len(dihedral)):
        for j in range(i+1, len(dihedral)):
            dihedral_neighbors[dihedral[i]].add(dihedral[j])
            dihedral_neighbors[dihedral[j]].add(dihedral[i])

CPU times: user 440 µs, sys: 0 ns, total: 440 µs
Wall time: 443 µs


In [94]:
%%time
test_neighbors = {atom: set() for atom in range(len(xlig))}
for dihedral in tops[0].topologies['Dihedrals']:
    for atom in dihedral:
        for i in range(len(dihedral)):
            if atom != dihedral[i]:
                test_neighbors[atom].add(dihedral[i])

CPU times: user 439 µs, sys: 75 µs, total: 514 µs
Wall time: 516 µs


In [96]:
%%time
improper_neighbors = {atom: set() for atom in range(len(xlig))}
for improper in tops[0].topologies['Impropers']:
    for i in range(len(improper)):
        for j in range(i+1, len(improper)):
            improper_neighbors[improper[i]].add(improper[j])
            improper_neighbors[improper[j]].add(improper[i])

CPU times: user 101 µs, sys: 17 µs, total: 118 µs
Wall time: 120 µs


In [53]:
topology = {"Angles": [[2, 1, 0],
                      [2, 3, 30],
                      [3, 2, 1],
                      [3, 2, 35],
                      [3, 2, 36],
                      [3, 4, 29],
                      [3, 30, 0],
                      [4, 3, 2],
                      [4, 3, 30],
                      [4, 5, 37]]}

atoms = set([atom for angle in topology['Angles'] for atom in angle])

angle_neighbors = {}
for i in atoms:
    angle_neighbors[i] = set()

for angle in topology['Angles']:
    for atom in angle:
        for i in range(len(angle)):
            if atom != angle[i]:
                angle_neighbors[atom].add(i)

In [51]:
topology = {"Angles": [[2, 1, 0],
                      [2, 3, 30],
                      [3, 2, 1],
                      [3, 2, 35],
                      [3, 2, 36],
                      [3, 4, 29],
                      [3, 30, 0],
                      [4, 3, 2],
                      [4, 3, 30],
                      [4, 5, 37]]}

atoms = set([atom for angle in topology['Angles'] for atom in angle])
atoms

{0, 1, 2, 3, 4, 5, 29, 30, 35, 36, 37}

In [39]:
test = {1, 2, 3}
test.append(4)

AttributeError: 'set' object has no attribute 'append'

In [126]:
coords = []
types = []
charges = []
with open("coords.txt") as f:
    dat = f.readlines()
    
for line in dat:
    raw = line.split()
    x = round(float(raw[3]),1)
    y = round(float(raw[4]),1)
    z = round(float(raw[5]),1)
    pos = [int(raw[0]), x,y,z]
    at = [int(raw[0]), int(raw[2])]
    charge = [int(raw[0]), float(raw[6])]
    charges.append(charge)
    types.append(at)
    coords.append(pos)

In [131]:
with open("clean_coords.txt", "w") as f:
    f.write("Coords\n\n")
    for pos in coords:
        f.write(f"{pos[0]}    {pos[1]} {pos[2]} {pos[3]}\n")
    f.write("\nTypes\n\n")
    for at in types:
        f.write(f"{at[0]}    {at[1]}\n")
    f.write("\nCharges\n\n")
    for charge in charges:
        f.write(f"{charge[0]}    {charge[1]}\n")
    f.write("\nSpecial Bond Counts\n\n")
    for atom in special_bond_counts:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bond_counts[atom]])}\n")
    f.write("\nSpecial Bonds\n\n")
    for atom in special_bonds:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bonds[atom]])}\n")

In [ ]:
with open("clean_types.txt", "w") as f:
    f.write()